In [1]:
import os
from pyhdf.SD import SD, SDC
import pandas as pd
import json
from processamento_dados import (selecao_area,
                                 controle_qualidade, porcentagem_valida,
                                 calc_media_espacial,
                                 calc_media_sazonal_1d, calc_media_sazonal_2d,
                                 calc_media_sazonal_anual_1d, calc_media_sazonal_anual_2d,
                                 convert_dicionario_1d_dataframe, convert_dicionario_2d_dataframe,
                                 preenchendo_dados,  calc_soma_deteccoes_coluna)
from plot_graficos import (plot_media_deteccoes_1d, plot_deteccoes_sazonais_2d, plot_coeficiente_extincao,
                           plot_multiplo_perfil_deteccao, plots_multiplos_perfil_ce_eoa,
                           plot_media_movel_deteccoes_eoa, plot_media_movel_deteccoes_2d)


In [2]:
# Abrindo arquivo com informações necessárias para o código
with open("info.json",'r') as f:
    info = json.load(f)
    f.close()

In [ ]:
# Variável contendo informações do produto de perfil de aerossóis troposféricos
info_produto = info['Produtos']['Tropospheric_Aerosol_Profile']

files = [f.path for f in os.scandir(info_produto['Diretório'])]

niveis_altitude = SD(files[0], SDC.READ).select('Altitude_Midpoint').get()[0]
perfil_media_sazonal_anual = {}
perfil_media_sazonal = {}
dados = {}
for area, info_area in list(info['Áreas_estudo'].items()):
    # Cria um dicionário vazio para cada área
    perfil_media_sazonal_anual[area] = {}
    perfil_media_sazonal[area] = {}
    dados[area] = {}
    for variavel, info_variavel in info_produto["Variáveis"].items():
        # Cria uma chave no dicionário para cada variável
        dados[area][variavel] = {}
        coordenadas = info_area['Coordenadas']
        for file in files:
            dado = SD(file, SDC.READ)
            # Abre cada arquivo listado
            # Seleciona dados de cada variável fazendo corte de área
            dado = selecao_area(dado, coordenadas, variavel)
            dado = controle_qualidade(dado, info_variavel['Limites_detecção'])
            # Adicionar mês ao dicionário apenas quando 80% dos dados passam no CQ
            if porcentagem_valida(dado) >= 0.8:
                dados[area][variavel][file[-12:-5]] = calc_media_espacial(dado)

        if variavel == 'Aerosol_Type':
            # Transforma array contendo classificação dos aerossóis em dataframe com colunas correspondendo a cada tipo de aerossol
            dados[area][variavel] = convert_dicionario_2d_dataframe(dados[area][variavel], info_variavel)
            # Salvando df com médias sazonais para cada ano e tipo de aerossol na chave da variável de detecção de aerossóis por tipo
            perfil_media_sazonal_anual[area][variavel] = calc_media_sazonal_anual_2d(dados[area][variavel])
            perfil_media_sazonal[area][variavel] = calc_media_sazonal_2d(perfil_media_sazonal_anual[area][variavel])
        else:
            # Salvando df com as médias sazonais para cada ano de cada variável de perfil único na chave da variável
            perfil_media_sazonal_anual[area][variavel] = calc_media_sazonal_anual_1d(dados[area][variavel])
            # Salvando df com as médias sazonais do período para cada variável de perfil único na chave da variável
            perfil_media_sazonal[area][variavel] = calc_media_sazonal_1d(perfil_media_sazonal_anual[area][variavel])
        
        if 'Ext' not in variavel:
            # Calcula a soma das detecções para a coluna
            dados[area][variavel] = calc_soma_deteccoes_coluna(dados[area][variavel])
            # Tranformando dicionário em df
            dados[area][variavel] = convert_dicionario_1d_dataframe(dados[area][variavel])
            dados[area][variavel] = preenchendo_dados(dados[area][variavel])

    dados[area]['AOD'] = pd.concat({variavel: valor for variavel, valor in dados[area].items() if 'AOD' in variavel}, axis=1).droplevel(1, axis=1)


In [ ]:
# Plots multiplas áreas
info_variaveis = info['Produtos']['Tropospheric_Aerosol_Profile']['Variáveis']
plots_multiplos_perfil_ce_eoa(perfil_media_sazonal, info_variaveis, niveis_altitude)
plot_multiplo_perfil_deteccao(perfil_media_sazonal, info_variaveis['Samples_Aerosol_Detected_Accepted'], niveis_altitude)

In [ ]:
# Plots individuais por área
for area in dados.keys():
    plot_media_deteccoes_1d(perfil_media_sazonal[area]['Samples_Aerosol_Detected_Accepted'].copy().T, niveis_altitude, area)
    informacoes_variaveis = info['Produtos']['Tropospheric_Aerosol_Profile']['Variáveis']
    plot_deteccoes_sazonais_2d(perfil_media_sazonal[area]['Aerosol_Type'].copy().T, niveis_altitude, area, informacoes_variaveis['Aerosol_Type'])
    plot_coeficiente_extincao(perfil_media_sazonal[area].copy(), informacoes_variaveis, niveis_altitude, area)   
    plot_media_movel_deteccoes_eoa(dados[area]["Samples_Aerosol_Detected_Accepted"].copy(), dados[area]["AOD_Mean"].copy(), area)
    plot_media_movel_deteccoes_2d(dados[area]['Aerosol_Type'].copy(), info_variavel, area)
